                   Perturbation for spins in magnetic field

1. We already have the zeeman interaction cal
2. H0 = (g_e*bohr_mag/h_cut) * B * S
#### Taking z direction as the direction of applied magnetic field
3. for gs = 2 and spins present in their eigen states, uo or down
4. E_plus = + bohr_mag * Bz;      and     E_minus = - bohr_mag * Bz;

#### Let's add the perturbation through magnetic field
1. Suppose we are in up state, then add small m.f. Bx, s.t. Bx << Bz ;
2. So what will be the change in energy of the eigen states ?
3. perturbation potential: V1 = (g_e*bohr_mag/h_cut) * Bx * Sx
4. Note that: only up and down are the spin eigen_states that we would need in 2nd order corr

In [2]:
import numpy as np

In [88]:
class spin_perturbation:
    def __init__(self,Bz, Bx):
        self.Bz = Bz
        self.Bx = Bx
        self.h_cut = 1
        self.m_e = 1
        self.e_charge = 1
        self.g_e = 2.002319
        self.bohr_magneton = self.e_charge*self.h_cut/(2*self.m_e)
        self.gyro_con = (self.g_e/self.h_cut)*self.bohr_magneton
        
    def spin_function(self):
        spin_up = np.array([[1], [0]])   # +z
        spin_down = np.array([[0],[1]])  # -z
        spin_right = np.array([[1],[1]])   # +x
        spin_left = np.array([[1], [-1]])  # -x
        spin_out = np.array([[1],[1j]])    # +y
        spin_in= np.array([[1], [-1j]])    #-y

        spin_dict = {"spin_up": spin_up, "spin_down":spin_down, "spin_right":spin_right,
                    "spin_left":spin_left, "spin_out":spin_out, "spin_in":spin_in}

        return spin_dict
    
    def spin_operators(self):
        sz = np.array([[1/2, 0], [0, -1/2]])
        s_plus = np.array([[0, 1/2], [0, 0]])
        s_minus = np.array([[0, 0], [1/2, 0]])
        sx = np.array([[0 ,1/2], [1/2, 0]])
        sy = np.array([[0, -1j/2], [1j/2, 0]])

        operate = {"sz":sz, "s_plus":s_plus, "s_minus":s_minus, "sx":sx, "sy":sy}

        return operate
    
    def potential(self):
        operate = self.spin_operators()
        sz = operate["sz"]
        sx = operate["sx"]
        V0 = (self.g_e*self.bohr_magneton/self.h_cut)*self.Bz*sz
        V1 = (self.g_e*self.bohr_magneton/self.h_cut)*self.Bx*sx

        pot = {"unperturbed":V0, "perturbed":V1}

        return pot
 
    def unperturbed_ene_cal(self, prepared_state):
        spin_states = self.spin_function()
        spin_up = spin_states["spin_up"]
        spin_down = spin_states["spin_down"]
        operate = self.spin_operators()
        sz = operate["sz"]

        e_up = spin_up.T @ sz @ spin_up
        e_down = spin_down.T @ sz @ spin_down

        zeeman_energies = {"E0_up": e_up, "E0_down":e_down}

        # for the given state
        e = prepared_state.T @ sz @ prepared_state
    
        return np.squeeze(e)
    
    def first_order_correction(self, spin_state):
        pot = self.potential()
        V1 = pot ["perturbed"]
        E_1dash = spin_state.T @ V1 @ spin_state

        return np.squeeze(E_1dash)
    
    def second_order_correction(self, prepared_state):
        E_2dash = 0
        spin_state_list = self.spin_function()
        eigen_states = [spin_state_list["spin_up"], spin_state_list["spin_down"]]
        pot = self.potential()
        V1 = pot["perturbed"]
        
        for other_state in eigen_states:
            if np.array_equal(other_state, prepared_state):
                E_2dash += 0
            else:
                eig_ene_diff = self.unperturbed_ene_cal(prepared_state) - self.unperturbed_ene_cal(other_state)
                e = np.square( prepared_state.T @ V1 @ other_state ) / eig_ene_diff
                E_2dash += e
        
        return np.squeeze(E_2dash)

In [100]:
Bz = 2
Bx = 0.5
method = spin_perturbation(Bz, Bx)

In [101]:
spin_states = method.spin_function()
prepared_state = spin_states["spin_up"]

In [102]:
E0 = method.unperturbed_ene_cal(prepared_state)
E_1dash = method.first_order_correction(prepared_state)
E_2dash = method.second_order_correction(prepared_state)
E = E0 + E_1dash + E_2dash

In [103]:
print(f"Unperturbed energy:{E0}\n1st order correction to energy:{E_1dash} \n2nd order correction:{E_2dash}")

Unperturbed energy:0.5
1st order correction to energy:0.0 
2nd order correction:0.06264502152751562
